### Imports

In [1]:
# Remove unwanted warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data extraction and management
import polars as pl
import numpy as np
import yfinance as yf

# Feature Engineering
from sklearn.preprocessing import StandardScaler

# Machine Learning
from sklearn.cluster import KMeans
from sklearn import metrics
from kneed import KneeLocator

# Cointegration and Statistics
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm

# Reporting visualization
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

## Data Extraction 

In [2]:
# Set Data Extraction parameters for trading with Polars
start_date = "2019-01-01"  # Start date for historical data
end_date = "2024-06-01"    # End date for historical data (adjust as needed)

# File paths for storing and retrieving data
file_name = "data/raw_data_etf.csv"  # Path for ETF data
file_name_coint = "data/raw_data_coint_pairs.csv"  # Path for cointegrated pairs data

# Flags for data loading
load_existing = True     # Set to True to load existing ETF data from file
load_coint_pairs = False   # Set to True to load existing cointegrated pairs data

# Note: When using Polars for trading:
# - Ensure date parsing is handled correctly when reading/writing CSV files
# - Consider using Polars' memory-efficient data types for large datasets
# - Leverage Polars' fast I/O operations for reading/writing data
# - Utilize Polars' vectorized operations for quick data manipulations

### Get Symbols

In [5]:
import yfinance as yf  # Import the yfinance library for financial data
import polars as pl    # Import the Polars library for DataFrame operations
import pandas as pd    # Import the pandas library for data manipulation

# Variable to determine whether to load existing data or fetch new data
load_existing = True

# Function to fetch NASDAQ symbols
def get_nasdaq_symbols():
    # URL of the NASDAQ listed symbols file
    url = 'ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt'
    # Read the symbols data from the URL using pandas
    symbols = pd.read_csv(url, sep='|')
    # Drop rows with any missing values
    symbols = symbols.dropna()
    # Set the 'Symbol' column as the index of the DataFrame
    symbols.set_index('Symbol', inplace=True)
    # Return the DataFrame containing the symbols
    return symbols

# Function to load existing symbols from a file
def load_existing_symbols(file_path):
    # Read the symbols from a CSV file
    existing_symbols = pd.read_csv(file_path)
    # Set the 'Symbol' column as the index of the DataFrame
    existing_symbols.set_index('Symbol', inplace=True)
    # Return the DataFrame containing the existing symbols
    return existing_symbols

# Path to the file where existing symbols are stored
existing_symbols_file = 'data/existing_symbols.csv'

# Check if we should load existing data or fetch new data
if load_existing:
    # Load existing symbols from the file
    existing_symbols = load_existing_symbols(existing_symbols_file)
    # Ensure symbols variable is defined when loading existing data
    symbols = existing_symbols.copy()
else:
    # Fetch the NASDAQ symbols using the defined function
    symbols = get_nasdaq_symbols()
    # Filter the symbols to include only those that are ETFs (ETF == 'Y') and belong to the NASDAQ Global Market (Market Category == 'G')
    symbols = symbols[(symbols['ETF'] == 'Y') & (symbols['Market Category'] == 'G')]
    # Convert the filtered symbols DataFrame index to a list of symbol strings
    new_symbols = list(symbols.index.values)

    # Compare new symbols with existing symbols (if they exist)
    try:
        existing_symbols = load_existing_symbols(existing_symbols_file)
        existing_symbols_list = list(existing_symbols.index.values)

        if new_symbols != existing_symbols_list:
            # If there are new or updated symbols, replace the existing symbols
            symbols.to_csv(existing_symbols_file)
            print("Symbols updated.")
        else:
            print("No new symbols.")
    except FileNotFoundError:
        # If the existing symbols file does not exist, save the new symbols
        symbols.to_csv(existing_symbols_file)
        print("Symbols saved.")

    # Print the list of new symbols
    print(new_symbols)

# Print the length of the symbols DataFrame
print(len(symbols))


644


In [6]:
print(len(symbols))

644


In [7]:
import yfinance as yf  
import pandas as pd   

# Ensure symbols is a list of strings, somehow errored because it was not in string format
if isinstance(symbols, pd.DataFrame):
    symbols = symbols.index.tolist()

# Fetch adjusted close prices for the symbols from Yahoo Finance
data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']

# Save the data to a CSV file
data.to_csv(file_name)

print(f"Data saved to {file_name}")


[*********************100%%**********************]  644 of 644 completed


19 Failed downloads:
['QQQT', 'IBTP', 'METD', 'BMDL', 'EVSD', 'AIPI', 'IBGA', 'TMET', 'GLOW', 'QBUF', 'METU', 'IBGK', 'BGRO', 'BRHY', 'QXQ', 'BSCY', 'FCTE', 'BSJW', 'BELT']: Exception("%ticker%: Data doesn't exist for startDate = 1546318800, endDate = 1717214400")



Data saved to data/raw_data_etf.csv


In [8]:
import polars as pl

# Load (or re-load for consistency) Data and remove features with NaN's
data = pl.read_csv(file_name)

# Filter columns that have all null values
data = data.filter(~pl.all_horizontal(pl.all().is_null()))

# Replace null values with zero for each column explicitly
data = data.with_columns(
    [pl.col(column).fill_null(0) for column in data.columns]
)

# Print the shape of the dataset
print("Shape:", data.shape)

# Check for any null values and print a boolean
has_null_values = data.null_count().sum_horizontal().sum() > 0
print("Null Values:", has_null_values)

# Display the first 50 rows to verify
data


Shape: (1363, 645)
Null Values: False


Date,AADR,AAPB,AAPD,AAPU,AAXJ,ABCS,ACWI,ACWX,AGMI,AGNG,AGZD,AIA,AIPI,AIQ,AIRL,AIRR,ALTY,AMDL,AMDS,AMID,AMZD,AMZU,AMZZ,ANGL,AOTG,AQWA,ARVR,ASET,AUMI,AVXC,BABX,BBH,BDGS,BEEZ,BELT,BGRN,…,VIGI,VMBS,VMOT,VNQI,VONE,VONG,VONV,VPLS,VRIG,VSDA,VSMV,VTC,VTHR,VTIP,VTWG,VTWO,VTWV,VWOB,VXUS,VYMI,WABF,WBND,WCBR,WCLD,WEEI,WGMI,WINC,WISE,WNDY,WOOD,WRND,WTBN,XBIL,XFIX,XT,YLDE,ZZZ
str,f64,str,str,str,f64,str,f64,f64,str,f64,f64,f64,str,f64,str,f64,f64,str,str,str,str,str,str,f64,str,str,str,f64,str,str,str,f64,str,str,str,f64,…,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,str,f64,str,str,f64,str,str,str,str,f64,f64,str
"""2019-01-02 00:00:00""",37.611717,"""0""","""0""","""0""",57.365265,"""0""",57.852905,35.92952,"""0""",18.078405,18.813507,49.291367,"""0""",12.734488,"""0""",21.639994,8.677715,"""0""","""0""","""0""","""0""","""0""","""0""",20.298044,"""0""","""0""","""0""",22.040136,"""0""","""0""","""0""",109.191826,"""0""","""0""","""0""",43.900227,…,49.100071,45.093941,22.233307,42.919483,104.997383,31.99737,43.003788,"""0""",20.79158,25.126345,23.901335,67.175751,104.828293,39.935486,117.246964,50.260662,84.563377,57.170956,40.046921,44.070274,"""0""",70.805138,"""0""","""0""","""0""","""0""",0.0,"""0""","""0""",51.865044,"""0""","""0""","""0""","""0""",31.699694,23.761587,"""0"""
"""2019-01-03 00:00:00""",37.192055,"""0""","""0""","""0""",56.010471,"""0""",56.869751,35.594372,"""0""",18.043736,18.773775,47.861721,"""0""",12.401687,"""0""",21.451563,8.691308,"""0""","""0""","""0""","""0""","""0""","""0""",20.32082,"""0""","""0""","""0""",21.988035,"""0""","""0""","""0""",109.97773,"""0""","""0""","""0""",43.900227,…,48.399124,45.277603,22.203083,42.985466,102.711014,31.044071,42.305408,"""0""",20.800022,24.62929,23.61002,67.293114,102.631538,40.043842,115.129044,49.30814,84.38298,57.323956,39.596558,43.921246,"""0""",70.721489,"""0""","""0""","""0""","""0""",0.0,"""0""","""0""",51.63876,"""0""","""0""","""0""","""0""",30.861732,23.422525,"""0"""
"""2019-01-04 00:00:00""",38.637577,"""0""","""0""","""0""",57.856266,"""0""",58.682724,36.694336,"""0""",18.67058,18.869135,49.552124,"""0""",12.900888,"""0""",22.145788,8.808433,"""0""","""0""","""0""","""0""","""0""","""0""",20.632036,"""0""","""0""","""0""",22.378813,"""0""","""0""","""0""",115.390671,"""0""","""0""","""0""",43.982491,…,49.963417,45.12019,22.258949,43.925674,106.126762,32.249981,43.498859,"""0""",20.816904,25.235594,24.027613,67.154816,105.91288,40.027168,119.461159,51.134193,86.574875,57.469261,40.888126,45.160465,"""0""",70.554146,"""0""","""0""","""0""","""0""",0.0,"""0""","""0""",53.449055,"""0""","""0""","""0""","""0""",31.918703,23.999838,"""0"""
"""2019-01-07 00:00:00""",39.094547,"""0""","""0""","""0""",58.201794,"""0""",59.043526,36.745903,"""0""",19.003744,18.80954,49.857834,"""0""",13.145594,"""0""",22.502819,8.98833,"""0""","""0""","""0""","""0""","""0""","""0""",20.912907,"""0""","""0""","""0""",22.569866,"""0""","""0""","""0""",118.102043,"""0""","""0""","""0""",43.982491,…,50.006168,45.067711,22.428375,44.181335,107.054161,32.566967,43.737545,"""0""",20.804249,25.426775,24.24408,67.191696,106.795273,40.010498,122.378128,52.096012,87.702347,57.668144,41.015591,45.199677,"""0""",70.916679,"""0""","""0""","""0""","""0""",0.0,"""0""","""0""",53.783962,"""0""","""0""","""0""","""0""",32.404335,24.228937,"""0"""
"""2019-01-08 00:00:00""",39.365002,"""0""","""0""","""0""",58.356365,"""0""",59.476467,36.98653,"""0""",19.02878,18.908875,49.722961,"""0""",13.272839,"""0""",22.790424,9.078927,"""0""","""0""","""0""","""0""","""0""","""0""",21.049541,"""0""","""0""","""0""",22.846022,"""0""","""0""","""0""",119.781921,"""0""","""0""","""0""",43.956505,…,50.373737,45.058949,22.501638,44.527733,108.027451,32.984028,44.055794,"""0""",20.816904,25.426775,24.402819,67.234428,107.925804,40.002174,124.361298,52.890549,89.100479,57.668144,41.312984,45.411446,"""0""",71.056122,"""0""","""0""","""0""","""0""",0.0,"""0""","""0""",

## Data Clean up

In [28]:
import polars as pl

def clean_data(df):
    # Function to try converting to float
    def try_float(s):
        try:
            return float(s)
        except ValueError:
            return s  # Return the original value if it can't be converted

    # Identify columns that are strings
    string_cols = [col for col, dtype in df.schema.items() if dtype == pl.String]
    
    for col in string_cols:
        # Try to convert the column to float
        df = df.with_columns(pl.col(col).apply(try_float))
        
        # Check if all values in the column are now float
        if df[col].dtype == pl.Float64:
            print(f"Column {col} successfully converted to Float64.")
        else:
            print(f"Column {col} contains non-numeric values and will remain as its original type.")

    return df

# Clean the data
cleaned_data = clean_data(data)

# Print schema to verify changes
print("\nUpdated Schema:")
print(cleaned_data.schema)

# Print the first few rows of the cleaned data
print("\nFirst few rows of cleaned data:")
print(cleaned_data.head())

# Count of float columns
float_cols = [col for col, dtype in cleaned_data.schema.items() if dtype == pl.Float64]
print(f"\nNumber of Float64 columns: {len(float_cols)}")

/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df = df.with_columns(pl.col(col).apply(try_float))
/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col(col).apply(try_float))
/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df = df.with_columns(pl.col(col).apply(try_float))
/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df 

Column Date contains non-numeric values and will remain as its original type.
Column AAPB successfully converted to Float64.
Column AAPD successfully converted to Float64.
Column AAPU successfully converted to Float64.
Column ABCS successfully converted to Float64.
Column AGMI successfully converted to Float64.
Column AIPI successfully converted to Float64.
Column AIRL successfully converted to Float64.
Column AMDL successfully converted to Float64.
Column AMDS successfully converted to Float64.
Column AMID successfully converted to Float64.
Column AMZD successfully converted to Float64.
Column AMZU successfully converted to Float64.
Column AMZZ successfully converted to Float64.
Column AOTG successfully converted to Float64.
Column AQWA successfully converted to Float64.
Column ARVR successfully converted to Float64.
Column AUMI successfully converted to Float64.
Column AVXC successfully converted to Float64.
Column BABX successfully converted to Float64.
Column BDGS successfully conv

/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col(col).apply(try_float))
/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df = df.with_columns(pl.col(col).apply(try_float))
/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col(col).apply(try_float))
/var/folders/4w/6x3yk1r13xvcwchd72ftlnsc0000gn/T/ipykernel_38198/3318089774.py:16: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df 

In [35]:
cleaned_data

Date,AADR,AAPB,AAPD,AAPU,AAXJ,ABCS,ACWI,ACWX,AGMI,AGNG,AGZD,AIA,AIPI,AIQ,AIRL,AIRR,ALTY,AMDL,AMDS,AMID,AMZD,AMZU,AMZZ,ANGL,AOTG,AQWA,ARVR,ASET,AUMI,AVXC,BABX,BBH,BDGS,BEEZ,BELT,BGRN,…,VIGI,VMBS,VMOT,VNQI,VONE,VONG,VONV,VPLS,VRIG,VSDA,VSMV,VTC,VTHR,VTIP,VTWG,VTWO,VTWV,VWOB,VXUS,VYMI,WABF,WBND,WCBR,WCLD,WEEI,WGMI,WINC,WISE,WNDY,WOOD,WRND,WTBN,XBIL,XFIX,XT,YLDE,ZZZ
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2019-01-02 00:00:00""",37.611717,0.0,0.0,0.0,57.365265,0.0,57.852905,35.92952,0.0,18.078405,18.813507,49.291367,0.0,12.734488,0.0,21.639994,8.677715,0.0,0.0,0.0,0.0,0.0,0.0,20.298044,0.0,0.0,0.0,22.040136,0.0,0.0,0.0,109.191826,0.0,0.0,0.0,43.900227,…,49.100071,45.093941,22.233307,42.919483,104.997383,31.99737,43.003788,0.0,20.79158,25.126345,23.901335,67.175751,104.828293,39.935486,117.246964,50.260662,84.563377,57.170956,40.046921,44.070274,0.0,70.805138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.865044,0.0,0.0,0.0,0.0,31.699694,23.761587,0.0
"""2019-01-03 00:00:00""",37.192055,0.0,0.0,0.0,56.010471,0.0,56.869751,35.594372,0.0,18.043736,18.773775,47.861721,0.0,12.401687,0.0,21.451563,8.691308,0.0,0.0,0.0,0.0,0.0,0.0,20.32082,0.0,0.0,0.0,21.988035,0.0,0.0,0.0,109.97773,0.0,0.0,0.0,43.900227,…,48.399124,45.277603,22.203083,42.985466,102.711014,31.044071,42.305408,0.0,20.800022,24.62929,23.61002,67.293114,102.631538,40.043842,115.129044,49.30814,84.38298,57.323956,39.596558,43.921246,0.0,70.721489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.63876,0.0,0.0,0.0,0.0,30.861732,23.422525,0.0
"""2019-01-04 00:00:00""",38.637577,0.0,0.0,0.0,57.856266,0.0,58.682724,36.694336,0.0,18.67058,18.869135,49.552124,0.0,12.900888,0.0,22.145788,8.808433,0.0,0.0,0.0,0.0,0.0,0.0,20.632036,0.0,0.0,0.0,22.378813,0.0,0.0,0.0,115.390671,0.0,0.0,0.0,43.982491,…,49.963417,45.12019,22.258949,43.925674,106.126762,32.249981,43.498859,0.0,20.816904,25.235594,24.027613,67.154816,105.91288,40.027168,119.461159,51.134193,86.574875,57.469261,40.888126,45.160465,0.0,70.554146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.449055,0.0,0.0,0.0,0.0,31.918703,23.999838,0.0
"""2019-01-07 00:00:00""",39.094547,0.0,0.0,0.0,58.201794,0.0,59.043526,36.745903,0.0,19.003744,18.80954,49.857834,0.0,13.145594,0.0,22.502819,8.98833,0.0,0.0,0.0,0.0,0.0,0.0,20.912907,0.0,0.0,0.0,22.569866,0.0,0.0,0.0,118.102043,0.0,0.0,0.0,43.982491,…,50.006168,45.067711,22.428375,44.181335,107.054161,32.566967,43.737545,0.0,20.804249,25.426775,24.24408,67.191696,106.795273,40.010498,122.378128,52.096012,87.702347,57.668144,41.015591,45.199677,0.0,70.916679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.783962,0.0,0.0,0.0,0.0,32.404335,24.228937,0.0
"""2019-01-08 00:00:00""",39.365002,0.0,0.0,0.0,58.356365,0.0,59.476467,36.98653,0.0,19.02878,18.908875,49.722961,0.0,13.272839,0.0,22.790424,9.078927,0.0,0.0,0.0,0.0,0.0,0.0,21.049541,0.0,0.0,0.0,22.846022,0.0,0.0,0.0,119.781921,0.0,0.0,0.0,43.956505,…,50.373737,45.058949,22.501638,44.527733,108.027451,32.984028,44.055794,0.0,20.816904,25.426775,24.402819,67.234428,107.925804,40.002174,124.361298,52.890549,89.100479,57.668144,41.312984,45.411446,0.0,71.056122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.290848,0.0,0.0,0.0,0.0,32.58527,24.250929,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024-05-24 00:00:00""",63.126698,20.99,20.237186,26.952614,71.892708,26.473419,110.770073,53.762825,28.23,29.647701,22.070314,66.198334,0.0,34.358345,25.25,73.17453,11.20698,17.66,12.5176,32.984001,14.392092,33.047562,26.098,28.301798,38.110001,17.668236,36.518539,30.676825,31.799999,51.826969,16.190001,168.050003,27.99,29.853001,0.0,46.1031,…,80.55307,44.700729,25.639999,41.759998,239.539413,88.926918,77.127655,75.033058,25.006441,48.683243,44.657619,74.986801,233.913132,4

## Feature Engineering

In [37]:
# import polars as pl
# import numpy as np

# # Assuming 'cleaned_data' is your DataFrame
# float_columns = [col for col, dtype in zip(cleaned_data.columns, cleaned_data.dtypes) if dtype == pl.Float64 and col != 'Date']

# # Calculate percentage change, mean returns, and volatility for each column
# returns = []
# volatility = []
# for col in float_columns:
#     pct_change = cleaned_data.select(pl.col(col)).with_columns(pl.col(col).pct_change().alias("pct_change"))
#     returns.append(pct_change.select(pl.mean("pct_change")).to_numpy()[0, 0] * 255)  # Annualized return
#     volatility.append(pct_change.select(pl.std("pct_change")).to_numpy()[0, 0] * np.sqrt(255))  # Annualized volatility

# # Create DataFrame with Returns and Volatility information
# df_returns = pl.DataFrame({
#     'ETF': float_columns,
#     'Returns': returns,
#     'Volatility': volatility
# })

# print("DataFrame with Returns and Volatility:")
# print(df_returns.tail())


DataFrame with Returns and Volatility:
shape: (5, 3)
┌──────┬──────────┬────────────┐
│ ETF  ┆ Returns  ┆ Volatility │
│ ---  ┆ ---      ┆ ---        │
│ str  ┆ f64      ┆ f64        │
╞══════╪══════════╪════════════╡
│ XBIL ┆ NaN      ┆ NaN        │
│ XFIX ┆ NaN      ┆ NaN        │
│ XT   ┆ 0.139008 ┆ 0.227996   │
│ YLDE ┆ 0.142387 ┆ 0.181442   │
│ ZZZ  ┆ NaN      ┆ NaN        │
└──────┴──────────┴────────────┘


In [40]:
import yfinance as yf

data_pd = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
print(data_pd.head())

[*********************100%%**********************]  644 of 644 completed


19 Failed downloads:
['QQQT', 'IBTP', 'METD', 'BMDL', 'EVSD', 'AIPI', 'IBGA', 'TMET', 'GLOW', 'QBUF', 'METU', 'IBGK', 'BGRO', 'BRHY', 'QXQ', 'BSCY', 'FCTE', 'BSJW', 'BELT']: Exception("%ticker%: Data doesn't exist for startDate = 1546318800, endDate = 1717214400")



                          AADR  AAPB  AAPD  AAPU       AAXJ  ABCS       ACWI  \
Date                                                                           
2019-01-02 00:00:00  37.611729   NaN   NaN   NaN  57.365269   NaN  57.852905   
2019-01-03 00:00:00  37.192059   NaN   NaN   NaN  56.010460   NaN  56.869747   
2019-01-04 00:00:00  38.637577   NaN   NaN   NaN  57.856266   NaN  58.682728   
2019-01-07 00:00:00  39.094547   NaN   NaN   NaN  58.201786   NaN  59.043522   
2019-01-08 00:00:00  39.365005   NaN   NaN   NaN  58.356361   NaN  59.476471   

                          ACWX  AGMI       AGNG  ...  WISE  WNDY       WOOD  \
Date                                             ...                          
2019-01-02 00:00:00  35.929516   NaN  18.078400  ...   NaN   NaN  51.865044   
2019-01-03 00:00:00  35.594376   NaN  18.043736  ...   NaN   NaN  51.638760   
2019-01-04 00:00:00  36.694340   NaN  18.670578  ...   NaN   NaN  53.449059   
2019-01-07 00:00:00  36.745895   NaN  19.00

In [42]:

# Removing columns with any NaN values
data_pd.dropna(axis=1, inplace=True)

# Checking the shape and null values after cleaning
print("Shape after dropping NaNs: ", data_pd.shape)
print("Null Values after dropping NaNs: ", data_pd.isnull().values.any())
data_pd.head()

Shape after dropping NaNs:  (1363, 308)
Null Values after dropping NaNs:  False


,AADR,AAXJ,ACWI,ACWX,AGNG,AGZD,AIA,AIQ,AIRR,ALTY,...,VTWG,VTWO,VTWV,VWOB,VXUS,VYMI,WBND,WOOD,XT,YLDE
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00,37.611729,57.365269,57.852905,35.929516,18.078400,18.813513,49.291367,12.734488,21.639992,8.677716,...,117.246964,50.260658,84.563393,57.170990,40.046917,44.070267,70.805153,51.865044,31.699688,23.761583
2019-01-03 00:00:00,37.192059,56.010460,56.869747,35.594376,18.043736,18.773775,47.861717,12.401689,21.451567,8.691305,...,115.129021,49.308140,84.382988,57.323944,39.596565,43.921249,70.721489,51.638760,30.861732,23.422527
2019-01-04 00:00:00,38.637577,57.856266,58.682728,36.694340,18.670578,18.869139,49.552128,12.900889,22.145786,8.808433,...,119.461151,51.134201,86.574867,57.469261,40.888130,45.160461,70.554161,53.449059,31.918707,23.999846
2019-01-07 00:00:00,39.094547,58.201786,59.043522,36.745895,19.003744,18.809542,49.857838,13.145594,22.502819,8.988331,...,122.378143,52.096004,87.702370,57.668121,41.015587,45.199677,70.916702,53.783958,32.404343,24.228937
2019-01-08 00:00:00,39.365005,58.356361,59.476471,36.986523,19.028780,18.908867,49.722965,13.272840,22.790424,9.078921,...,124.361290,52.890545,89.100494,57.668121,41.312984,45.411430,71.056145,54.290840,32.585270,24.250929


In [43]:
import polars as pl
import numpy as np

# Convert Pandas DataFrame to Polars DataFrame, including resetting the index.
# This is done to convert the Date from the index to a regular column,
# which we don't need for our analysis, thus simplifying the DataFrame structure.
pl_df = pl.from_pandas(data_pd.reset_index())

# Identify columns with floating point data which are typically price data in financial datasets.
# We exclude the 'Date' column because time data is irrelevant for K-means clustering in this context.
# K-means clustering will be used to find patterns in price movements, not time series trends.
float_columns = [col for col, dtype in zip(pl_df.columns, pl_df.dtypes) if isinstance(dtype, pl.Float64) and col != 'Date']

# Initialize lists to store calculated annual returns and volatilities for each stock.
# These metrics are crucial for trading as they provide insights into the risk-return profile of stocks.
returns = []
volatility = []
for col in float_columns:
    # Calculate the percentage change, which is a common financial metric to assess stock performance.
    pct_change = pl_df.select(pl.col(col)).with_columns(pl.col(col).pct_change().alias("pct_change"))
    
    # Calculate and store annualized returns, which normalize returns over a year for comparability.
    # This is crucial for clustering as it standardizes data points, making the K-means algorithm more effective.
    annual_return = pct_change.select(pl.mean("pct_change")).to_numpy() * 255
    returns.append(annual_return[0, 0])  # Extract the scalar from the numpy array
    
    # Calculate and store annualized volatility, a measure of the price variability and thus risk.
    # Higher volatility stocks may cluster differently from lower volatility stocks in K-means.
    annual_volatility = pct_change.select(pl.std("pct_change")).to_numpy() * np.sqrt(255)
    volatility.append(annual_volatility[0, 0])  # Extract the scalar from the numpy array

# Create a new DataFrame containing the returns and volatility of each stock.
# This DataFrame is well-suited for K-means clustering to segment stocks into groups based on their
# risk-return profiles, which can inform investment strategies and risk management.
df_returns = pl.DataFrame({
    'ETF': float_columns,
    'Returns': returns,
    'Volatility': volatility
})

print("DataFrame with Returns and Volatility:")
print(df_returns)


DataFrame with Returns and Volatility:
shape: (308, 3)
┌──────┬───────────┬────────────┐
│ ETF  ┆ Returns   ┆ Volatility │
│ ---  ┆ ---       ┆ ---        │
│ str  ┆ f64       ┆ f64        │
╞══════╪═══════════╪════════════╡
│ AADR ┆ 0.127934  ┆ 0.233968   │
│ AAXJ ┆ 0.060713  ┆ 0.217516   │
│ ACWI ┆ 0.139888  ┆ 0.195757   │
│ ACWX ┆ 0.093164  ┆ 0.194469   │
│ AGNG ┆ 0.109739  ┆ 0.186009   │
│ …    ┆ …         ┆ …          │
│ VYMI ┆ 0.104908  ┆ 0.190829   │
│ WBND ┆ -0.135238 ┆ 0.320158   │
│ WOOD ┆ 0.117898  ┆ 0.254531   │
│ XT   ┆ 0.139008  ┆ 0.227996   │
│ YLDE ┆ 0.142387  ┆ 0.181442   │
└──────┴───────────┴────────────┘


In [46]:
import polars as pl
import numpy as np
from sklearn.preprocessing import StandardScaler

# Convert Polars DataFrame to NumPy array for scaling
data_for_scaling = df_returns.select(['Returns', 'Volatility']).to_numpy()

# Initialize and apply StandardScaler
# The scaler adjusts each feature to have zero mean and unit variance,
# which is vital for uniform analysis across different trading instruments,
# ensuring no single asset's characteristics dominate due to scale differences.
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_scaling)  # Use the correct data array

# Convert the scaled data back to a Polars DataFrame
# This maintains efficient data operations while ensuring comparability across assets.
scaled_df = pl.DataFrame(scaled_data, schema=['Scaled_Returns', 'Scaled_Volatility'])

# Assuming 'ETF' is a column in the original df_returns you want to keep
df_returns_scaled = df_returns.drop(['Returns', 'Volatility']).with_columns(scaled_df)

# Output the scaled DataFrame
# Scaled features are now ready for clustering or other statistical techniques
# that require standardized input to identify patterns in trading data effectively.
print(df_returns_scaled)


shape: (308, 3)
┌──────┬────────────────┬───────────────────┐
│ ETF  ┆ Scaled_Returns ┆ Scaled_Volatility │
│ ---  ┆ ---            ┆ ---               │
│ str  ┆ f64            ┆ f64               │
╞══════╪════════════════╪═══════════════════╡
│ AADR ┆ 0.261166       ┆ 0.194314          │
│ AAXJ ┆ -0.497126      ┆ 0.030127          │
│ ACWI ┆ 0.396008       ┆ -0.187024         │
│ ACWX ┆ -0.131066      ┆ -0.199882         │
│ AGNG ┆ 0.05591        ┆ -0.284313         │
│ …    ┆ …              ┆ …                 │
│ VYMI ┆ 0.001421       ┆ -0.236203         │
│ WBND ┆ -2.707533      ┆ 1.05449           │
│ WOOD ┆ 0.147955       ┆ 0.399533          │
│ XT   ┆ 0.386085       ┆ 0.134713          │
│ YLDE ┆ 0.424196       ┆ -0.329885         │
└──────┴────────────────┴───────────────────┘
